In [ ]:
import numpy as np
import scipy.linalg as spla
import scipy.sparse as sps
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = [9., 6.]

In [ ]:
from pymor.core.logger import set_log_levels

set_log_levels({'pymor.algorithms.gram_schmidt.gram_schmidt': 'ERROR'})

## Mass-spring-damper system

In [ ]:
n = 100
k = 1.
d = 0.5

M = sps.eye(n, format='csc')
mat = sps.diags([(n - 1) * [-1], n * [2], (n - 1) * [-1]], [-1, 0, 1], format='csc')
E = d * mat
K = k * mat

B = np.zeros((n, 1))
B[0, 0] = 1

C = np.zeros((1, n))
C[0, n//2] = 1

In [ ]:
from pymor.models.iosys import SecondOrderModel

In [ ]:
fom = SecondOrderModel.from_matrices(M, E, K, B, C)

## Poles

In [ ]:
poles_fom = fom.poles()
fig, ax = plt.subplots()
_ = ax.plot(poles_fom.real, poles_fom.imag, '.')

## Bode plot

In [ ]:
w = np.logspace(-3, 0, 1000)
_ = fom.bode_plot(w)

## Magnitude plot

In [ ]:
_ = fom.mag_plot(w)

## Hankel singular values

In [ ]:
hsv = fom.to_lti().hsv()
plt.semilogy(range(1, 2 * n + 1), hsv, '.-')

# Second-order balanced truncation

In [ ]:
from pymor.reductors.sobt import SOBTReductor

In [ ]:
sobt = SOBTReductor(fom)

In [ ]:
r = 20
rom_sobt = sobt.reduce(r)

In [ ]:
err_sobt = fom - rom_sobt

## Poles

In [ ]:
poles_sobt = rom_sobt.poles()
fig, ax = plt.subplots()
_ = ax.plot(poles_sobt.real, poles_sobt.imag, '.')

## Bode plots

In [ ]:
fig, ax = plt.subplots(2, 1, squeeze=False, figsize=(9, 9), tight_layout=True)
_ = fom.bode_plot(w, ax=ax)
_ = rom_sobt.bode_plot(w, ax=ax)

## Error magnitude plot

In [ ]:
_ = err_sobt.mag_plot(w)

## Relative $\mathcal{H}_2$ error

In [ ]:
err_sobt.h2_norm() / fom.h2_norm()

## Relative $\mathcal{H}_\infty$ error

In [ ]:
err_sobt.hinf_norm() / fom.hinf_norm()

# Second-order reduced IRKA

In [ ]:
from pymor.reductors.sor_irka import SORIRKAReductor

In [ ]:
sorirka = SORIRKAReductor(fom)

In [ ]:
r = 20
rom_sorirka = sorirka.reduce(r, conv_crit='h2', irka_options={'conv_crit': 'h2'})

In [ ]:
err_sorirka = fom - rom_sorirka

## Poles

In [ ]:
poles_sorirka = rom_sorirka.poles()
fig, ax = plt.subplots()
_ = ax.plot(poles_sorirka.real, poles_sorirka.imag, '.')

## Bode plots

In [ ]:
fig, ax = plt.subplots(2, 1, squeeze=False, figsize=(9, 9), tight_layout=True)
_ = fom.bode_plot(w, ax=ax)
_ = rom_sorirka.bode_plot(w, ax=ax)

## Error magnitude plot

In [ ]:
_ = err_sorirka.mag_plot(w)

## Relative $\mathcal{H}_2$ error

In [ ]:
err_sorirka.h2_norm() / fom.h2_norm()

## Relative $\mathcal{H}_\infty$ error

In [ ]:
err_sorirka.hinf_norm() / fom.hinf_norm()